In [1]:
import pandas as pd
import numpy as np
import pytorch_lightning as pl
import torch
import selfies as sf

from rxitect.models.pchembl_val_predictor import PChEMBLValueRegressor
from rxitect.data.datasets import SelfiesQsarDataset, SelfiesQsarDataModule

/home/julius/miniconda3/envs/rxt-dev/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
df = pd.read_parquet("../data/processed/CHEMBL240_dataset.pq")

In [3]:
def safe_smiles_to_selfies(smiles):
    try:
        return sf.encoder(smiles)
    except sf.EncoderError:
        pass

In [4]:
df = df.dropna(subset=['pchembl value'])
df['selfies'] = [safe_smiles_to_selfies(smi) for smi in df['smiles']]

In [5]:
reg = PChEMBLValueRegressor(lr=0.05)

In [8]:
dm = SelfiesQsarDataModule(df)
trainer = pl.Trainer(accelerator="gpu", max_epochs=26)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [13]:
df['selfies']

index
1        [C][C][=N][C][=C][Branch2][Ring2][Branch2][O][...
5        [C][C][=C][C][=N][C][=C][Ring1][=Branch1][C][C...
7        [C][C][C][C][N][Branch1][Branch1][C][C][C][C][...
10       [C][C][=C][C][Branch2][Ring2][#C][N][C][C@@H1]...
11       [C][C][=C][N][=C][C][Branch2][Ring2][N][N][C][...
                               ...                        
25222    [C][C][=C][C][=C][Branch1][Ring2][S][Ring1][Br...
25223    [N][C][C][C][C][N][Branch1][S][C][C@H1][C][C][...
25224    [C][N][Branch2][Ring1][#Branch2][C][C@H1][C][C...
25226    [O][=C][Branch1][C][O][C][C][=C][C][=C][C][=C]...
25227    [O][=C][Branch2][Ring1][#C][C][C][C][C][N][C][...
Name: selfies, Length: 13936, dtype: object

In [9]:
trainer.fit(model=reg, datamodule=dm)

Missing logger folder: /home/julius/University/Thesis/rxitect/notebooks/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type        | Params
------------------------------------------
0 | criterion | MSELoss     | 0     
1 | embedding | Embedding   | 16.4 K
2 | encoder   | LSTMEncoder | 264 K 
3 | mlp       | MLP         | 16.6 K
------------------------------------------
297 K     Trainable params
0         Non-trainable params
297 K     Total params
1.189     Total estimated model params size (MB)


Epoch 0:   0%|                                                                                                                                                | 0/138 [00:00<?, ?it/s]

RuntimeError: Found dtype Double but expected Float